# Interface

See the article [implementing an interface in python](https://realpython.com/python-interface/) on realpython.org.

An interface is some abstraction that allows you to define what methods must necessarily be implemented in successors.

## Problem

Here is an attempt to explain a problem that is solved by abstract classes.

Suppose we need to implement two toy classes, one printing two strings separated by `+`, another separated by `-`. Let's try to solve this head-on.

In [1]:
class PlusConcatenation():
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def get_output(self):
        return f"{self.left}+{self.right}"

class MinusConcatenation():
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def get_output(self):
        return f"{self.left}-{self.right}"

print(
    "Plus concatenation",
    PlusConcatenation("hello", "world").get_output()
)
print(
    "Minus concatenation",
    MinusConcatenation("hello", "world").get_output()
)

Plus concatenation hello+world
Minus concatenation hello-world


Everything works fine, but we had to repeat the code:

```python
def __init__(self, left, right):
    self.left = left
    self.right = right
```

In this case, it's not a big problem. But repeating when there may be many more, and it may be repeated in more places - it will be difficult to maintain.

## Basic solution

That's where the OOP technique comes in - let's define a parent class that implements the common functionality of the inheritors, and each inheritor specifies its own functionality.

So here `__init__` is common for all `Concatenation` but `get_output` have to defined by inheritors.

In [53]:
class Concatenation():
    def __init__(self, left, right):
        self.left = left
        self.right = right
    def get_output(self):
        pass

class PlusConcatenation(Concatenation):
    def get_output(self):
        return f"{self.left}+{self.right}"
class MinusConcatenation(Concatenation):
    def get_output(self):
        return f"{self.left}-{self.right}"
print(
    "Plus concatenation",
    PlusConcatenation("hello", "world").get_output()
)
print(
    "Minus concatenation",
    MinusConcatenation("hello", "world").get_output()
)

Plus concatenation hello+world
Minus concatenation hello-world


Well, we got what we wanted.

But there are still some problems with these approaches, which are not really critical, but you have to keep them in mind. If inherit doesn't realise that it's functional, it can't be considered an implementation of the interface - but the syntax below allows you to create such objects, and then they will even be identified as subclasses of the parent class.

In the following cell we have created a `DummyConcatenation` which doesn't implement `get_output`. The problem is that it's still a subclass of the `DummyConcatenation' and its instances can be created freely.

In [76]:
class DummyConcatenation(Concatenation):
    pass

print(
    "issubclass(DummyConcatenation, Concatenation)",
    issubclass(DummyConcatenation, Concatenation)
)
print(
    'DummyConcatenation("hello", "world").get_output()',
    DummyConcatenation("hello", "world").get_output()
)

issubclass(DummyConcatenation, Concatenation) True
DummyConcatenation("hello", "world").get_output() None


## Redefining dunders

In [14]:
class MetaConcatenation(type):
    def __subclasscheck__(cls, subclass):
        return (
            hasattr(subclass, 'get_output') and
            callable(subclass.get_output)
        )
    def __instancecheck__(cls, instance):
        return cls.__subclasscheck__(type(instance))

In [28]:
class PlusConcatenation(MetaConcatenation):
    def get_output(self):
        return f"{self.left}+{self.right}"
class Dummy(MetaConcatenation):
    pass

In [29]:
print(
    "issubclass(PlusConcatenation, MetaConcatenation)",
    issubclass(PlusConcatenation, MetaConcatenation)
)
print(
    "issubclass(PlusConcatenation, Dummy)",
    issubclass(PlusConcatenation, Dummy)
)

issubclass(PlusConcatenation, MetaConcatenation) True
issubclass(PlusConcatenation, Dummy) False


In [30]:
print(
    "PlusConcatenation.__mro__",
    PlusConcatenation.__mro__
)
print("Dummy.__mro__", Dummy.__mro__)

PlusConcatenation.__mro__ (<class '__main__.PlusConcatenation'>, <class '__main__.MetaConcatenation'>, <class 'type'>, <class 'object'>)
Dummy.__mro__ (<class '__main__.Dummy'>, <class '__main__.MetaConcatenation'>, <class 'type'>, <class 'object'>)


## `abc` module

Для формальной реализации интерфейсов используется модуль `abc`

In [37]:
import abc
class Concatenation(metaclass=abc.ABCMeta):
    def __init__(self, left, right):
        self.left = left
        self.right = right
    @abc.abstractmethod
    def get_output(self, path: str, file_name: str):
        """Load in the data set"""
        raise NotImplementedError

In [41]:
class PlusConcatenation(Concatenation):
    def get_output(self):
        return f"{self.left}+{self.right}"
class DummyConcatenation(Concatenation):
    pass

In [49]:
PlusConcatenation("hello", "world").get_output()
try:
    DummyConcatenation("hello", "world")
except Exception as e:
    print(e)

Can't instantiate abstract class DummyConcatenation with abstract method get_output


In [45]:
issubclass(DummyConcatenation, Concatenation)

True